# Pipeline for RNA-Seq analysis

In [14]:
# Mark : Autogenerated with openBIS-ELN/LIMS (System Cell, Don't delete!)

In [15]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['font.family'] = 'Arial'

import seaborn as sns
sns.set(color_codes=True)

from pyCRAC.Parsers import GTF2
from collections import defaultdict
from sklearn.cluster import KMeans
from scipy.stats import pearsonr,spearmanr

from sg_ngs_tools import *
import obis_connector

%matplotlib inline


In [1]:
# Variable - File Name (System Cell, Don't delete!, Update if renamed)
fileName='RNA_pipeline.ipynb'
workspace='ngs/sander/phage_infection_time_course_e_coli_rnaseq'

In [17]:
# Variable : Parents used by the automaticaly generated result dataset (System Cell, Don't delete!)
resultDatasetParents=["20211105180528381-5936"]

### Connect to openBIS

In [18]:
import obis_connector
o = obis_connector.connect_to_openbis(url='http://openngs2.bio.ed.ac.uk:8443')

Using hardcoded url instead: http://openngs2.bio.ed.ac.uk:8443


### Datasets Information and Download

In [19]:
# Dataset ''20211105180528381-5936:
s0 = o.get_object('20211105164909360-5934')
#print(s0.summary)

ds0 = o.get_dataset('20211105180528381-5936')
#print(ds0.summary)

print(ds0.props.name)
# files0 = ds0.download(files=['original/DEFAULT/file1']) # to download selected, ds0.file_list lists present

files0 = ds0.download()

###################
# you can use files object to easily access the full file paths, for example:
# files0.path('filename')
# files0.paths lists all paths
# files0.glob('*.txt') lists paths matching glob pattern
# files0.help() to see commands help
###################

files0.paths

Processing of fastq files
Skipping the existing: RNASeq_data_analyses.ipynb
Skipping the existing: RNASeq_data_analyses.ipynb.html
Skipping the existing: a10_rep2_+T4_t15_1.fastq
Skipping the existing: a10_rep2_+T4_t15_2.fastq
Skipping the existing: a11_rep2_+T4_t20_1.fastq
Skipping the existing: a11_rep2_+T4_t20_2.fastq
Skipping the existing: a12_rep2_control_t20_1.fastq
Skipping the existing: a12_rep2_control_t20_2.fastq
Skipping the existing: a13_rep3_control_t-5_1.fastq
Skipping the existing: a13_rep3_control_t-5_2.fastq
Skipping the existing: a14_rep3_+T4_t2_1.fastq
Skipping the existing: a14_rep3_+T4_t2_2.fastq
Skipping the existing: a15_rep3_+T4_t5_1.fastq
Skipping the existing: a15_rep3_+T4_t5_2.fastq
Skipping the existing: a16_rep3_+T4_t10_1.fastq
Skipping the existing: a16_rep3_+T4_t10_2.fastq
Skipping the existing: a17_rep3_+T4_t20_1.fastq
Skipping the existing: a17_rep3_+T4_t20_2.fastq
Skipping the existing: a18_rep3_control_t20_1.fastq
Skipping the existing: a18_rep3_contr

['20211105180528381-5936/original/DEFAULT/RNASeq_data_analyses.ipynb',
 '20211105180528381-5936/original/DEFAULT/RNASeq_data_analyses.ipynb.html',
 '20211105180528381-5936/original/DEFAULT/a10_rep2_+T4_t15_1.fastq',
 '20211105180528381-5936/original/DEFAULT/a10_rep2_+T4_t15_2.fastq',
 '20211105180528381-5936/original/DEFAULT/a11_rep2_+T4_t20_1.fastq',
 '20211105180528381-5936/original/DEFAULT/a11_rep2_+T4_t20_2.fastq',
 '20211105180528381-5936/original/DEFAULT/a12_rep2_control_t20_1.fastq',
 '20211105180528381-5936/original/DEFAULT/a12_rep2_control_t20_2.fastq',
 '20211105180528381-5936/original/DEFAULT/a13_rep3_control_t-5_1.fastq',
 '20211105180528381-5936/original/DEFAULT/a13_rep3_control_t-5_2.fastq',
 '20211105180528381-5936/original/DEFAULT/a14_rep3_+T4_t2_1.fastq',
 '20211105180528381-5936/original/DEFAULT/a14_rep3_+T4_t2_2.fastq',
 '20211105180528381-5936/original/DEFAULT/a15_rep3_+T4_t5_1.fastq',
 '20211105180528381-5936/original/DEFAULT/a15_rep3_+T4_t5_2.fastq',
 '20211105180

### Process your data here

In [20]:
###### MAKE A VARIABLE THAT CONTAINS A LIST OF ALL THE FILES THAT YOU WANT TO UPLOAD #######
analysisfile = list()


In [21]:
%%bash
# Please only use scripts and annotations files from these folders so they will be tracked with the analysis
SCRIPTS=/homes2/openngs/openngs/scripts2
ANNOTATIONS=/homes2/openngs/openngs/annotation_files2

$SCRIPTS/check_dir_status.sh $SCRIPTS || exit 1
$SCRIPTS/check_dir_status.sh $ANNOTATIONS || exit 1



Checkig /homes2/openngs/openngs/scripts2
All content is recorded as:

## v.1.0.0...origin/v.1.0.0
commit 5f1c9cb1a751385062cefdf1a2bc7c298402cb08
Author: openngs2 <bio_rdm@ed.ac.uk>
Date:   Tue May 3 00:29:23 2022 +0100

    removed python2 checking

Checkig /homes2/openngs/openngs/annotation_files2
All content is recorded as:

## master...origin/master
commit 641054659eb96a75fca1a051a5f5e956c4b04dfc
Author: openngs2 <bio_rdm@ed.ac.uk>
Date:   Mon Jul 4 18:56:49 2022 +0100

    Fixed PROKKA GTF file


In [22]:
files0.paths

['20211105180528381-5936/original/DEFAULT/RNASeq_data_analyses.ipynb',
 '20211105180528381-5936/original/DEFAULT/RNASeq_data_analyses.ipynb.html',
 '20211105180528381-5936/original/DEFAULT/a10_rep2_+T4_t15_1.fastq',
 '20211105180528381-5936/original/DEFAULT/a10_rep2_+T4_t15_2.fastq',
 '20211105180528381-5936/original/DEFAULT/a11_rep2_+T4_t20_1.fastq',
 '20211105180528381-5936/original/DEFAULT/a11_rep2_+T4_t20_2.fastq',
 '20211105180528381-5936/original/DEFAULT/a12_rep2_control_t20_1.fastq',
 '20211105180528381-5936/original/DEFAULT/a12_rep2_control_t20_2.fastq',
 '20211105180528381-5936/original/DEFAULT/a13_rep3_control_t-5_1.fastq',
 '20211105180528381-5936/original/DEFAULT/a13_rep3_control_t-5_2.fastq',
 '20211105180528381-5936/original/DEFAULT/a14_rep3_+T4_t2_1.fastq',
 '20211105180528381-5936/original/DEFAULT/a14_rep3_+T4_t2_2.fastq',
 '20211105180528381-5936/original/DEFAULT/a15_rep3_+T4_t5_1.fastq',
 '20211105180528381-5936/original/DEFAULT/a15_rep3_+T4_t5_2.fastq',
 '20211105180

In [23]:
files0.glob("*.fastq")

['20211105180528381-5936/original/DEFAULT/a10_rep2_+T4_t15_1.fastq',
 '20211105180528381-5936/original/DEFAULT/a10_rep2_+T4_t15_2.fastq',
 '20211105180528381-5936/original/DEFAULT/a11_rep2_+T4_t20_1.fastq',
 '20211105180528381-5936/original/DEFAULT/a11_rep2_+T4_t20_2.fastq',
 '20211105180528381-5936/original/DEFAULT/a12_rep2_control_t20_1.fastq',
 '20211105180528381-5936/original/DEFAULT/a12_rep2_control_t20_2.fastq',
 '20211105180528381-5936/original/DEFAULT/a13_rep3_control_t-5_1.fastq',
 '20211105180528381-5936/original/DEFAULT/a13_rep3_control_t-5_2.fastq',
 '20211105180528381-5936/original/DEFAULT/a14_rep3_+T4_t2_1.fastq',
 '20211105180528381-5936/original/DEFAULT/a14_rep3_+T4_t2_2.fastq',
 '20211105180528381-5936/original/DEFAULT/a15_rep3_+T4_t5_1.fastq',
 '20211105180528381-5936/original/DEFAULT/a15_rep3_+T4_t5_2.fastq',
 '20211105180528381-5936/original/DEFAULT/a16_rep3_+T4_t10_1.fastq',
 '20211105180528381-5936/original/DEFAULT/a16_rep3_+T4_t10_2.fastq',
 '20211105180528381-59

## the pipeline for processing the raw fastq data

### STEP1: run the fastqc on all the fastq files.

In [87]:
%%bash
path_fastq="./20211105180528381-5936/original/DEFAULT/"
fastq_name=$(ls ./20211105180528381-5936/original/DEFAULT/*.fastq | grep -Eo 'a[0-9]+_.*t.*[0-9]+' | sort | uniq )
R1_file=$(ls ./20211105180528381-5936/original/DEFAULT/*.fastq |grep -Eo 'a[0-9]+_.*t.*[0-9]+_1'| sort | uniq )
R2_file=$(ls ./20211105180528381-5936/original/DEFAULT/*.fastq |grep -Eo 'a[0-9]+_.*t.*[0-9]+_2'| sort | uniq )
#echo $R1_file
for ID in $fastq_name
    do
    fastqc -o ./yan_results/fastqc_result/ -q -t 10 $path_fastq$ID".fastq"
    echo 'finished the analysis of' $path_fastq$ID".fastq"
    done
    

finished the analysis of ./20211105180528381-5936/original/DEFAULT/a10_rep2_+T4_t15_1.fastq
finished the analysis of ./20211105180528381-5936/original/DEFAULT/a10_rep2_+T4_t15_2.fastq
finished the analysis of ./20211105180528381-5936/original/DEFAULT/a11_rep2_+T4_t20_1.fastq
finished the analysis of ./20211105180528381-5936/original/DEFAULT/a11_rep2_+T4_t20_2.fastq
finished the analysis of ./20211105180528381-5936/original/DEFAULT/a12_rep2_control_t20_1.fastq
finished the analysis of ./20211105180528381-5936/original/DEFAULT/a12_rep2_control_t20_2.fastq
finished the analysis of ./20211105180528381-5936/original/DEFAULT/a13_rep3_control_t-5_1.fastq
finished the analysis of ./20211105180528381-5936/original/DEFAULT/a13_rep3_control_t-5_2.fastq
finished the analysis of ./20211105180528381-5936/original/DEFAULT/a14_rep3_+T4_t2_1.fastq
finished the analysis of ./20211105180528381-5936/original/DEFAULT/a14_rep3_+T4_t2_2.fastq
finished the analysis of ./20211105180528381-5936/original/DEFAULT

### STEP2: run the cutadapt on all the fastq files
( need to distinguish forward file and reverse file)

In [184]:
%%bash
cutadapt --h

cutadapt version 4.0

Copyright (C) 2010-2022 Marcel Martin <marcel.martin@scilifelab.se>

cutadapt removes adapter sequences from high-throughput sequencing reads.

Usage:
    cutadapt -a ADAPTER [options] [-o output.fastq] input.fastq

For paired-end reads:
    cutadapt -a ADAPT1 -A ADAPT2 [options] -o out1.fastq -p out2.fastq in1.fastq in2.fastq

Replace "ADAPTER" with the actual sequence of your 3' adapter. IUPAC wildcard
characters are supported. All reads from input.fastq will be written to
output.fastq with the adapter sequence removed. Adapter matching is
error-tolerant. Multiple adapter sequences can be given (use further -a
options), but only the best-matching adapter will be removed.

Input may also be in FASTA format. Compressed input and output is supported and
auto-detected from the file name (.gz, .xz, .bz2). Use the file name '-' for
standard input/output. Without the -o option, output is sent to standard output.

Citation:

Marcel Martin. Cutadapt removes adapter seque

                        Discard reads whose expected number of errors (computed
                        from quality values) exceeds ERRORS.
  --discard-trimmed, --discard
                        Discard reads that contain an adapter. Use also -O to
                        avoid discarding too many randomly matching reads.
  --discard-untrimmed, --trimmed-only
                        Discard reads that do not contain an adapter.
  --discard-casava      Discard reads that did not pass CASAVA filtering (header
                        has :Y:).

Output:
  --quiet               Print only error messages.
  --report {full,minimal}
                        Which type of report to print: 'full' or 'minimal'.
                        Default: full
  --json FILE           Dump report in JSON format to FILE
  -o FILE, --output FILE
                        Write trimmed reads to FILE. FASTQ or FASTA format is
                        chosen depending on input. Summary report is sent to
             

In [24]:
%%bash
adaptor1=GATCGGAAGAGCACACGTCTGAACTCCAGTCACGGATGACTATCTCGTATGCCGTCTTCTGCTTG
adaptor2=AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGTAGATCTCGGTGGTCGCCGTATCATT
fastq_name=$(ls ./20211105180528381-5936/original/DEFAULT/*.fastq | grep -Eo 'a[0-9]+_.*t.*[0-9]+_'  | sort | uniq )
path_fastq="./20211105180528381-5936/original/DEFAULT/"
for ID in $fastq_name
    do 
    R1_file=$path_fastq$ID"1.fastq"
    filter_R1=$path_fastq$ID"1_filter.fastq"
    R2_file=$path_fastq$ID"2.fastq"
    filter_R2=$path_fastq$ID"2_filter.fastq"
    cutadapt -a $adaptor1 -A $adaptor2 -o $filter_R1 -p $filter_R2 $R1_file $R2_file --cores=0 -m 50
    done
    

This is cutadapt 4.0 with Python 3.9.7
Command line parameters: -a GATCGGAAGAGCACACGTCTGAACTCCAGTCACGGATGACTATCTCGTATGCCGTCTTCTGCTTG -A AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGTAGATCTCGGTGGTCGCCGTATCATT -o ./20211105180528381-5936/original/DEFAULT/a10_rep2_+T4_t15_1_filter.fastq -p ./20211105180528381-5936/original/DEFAULT/a10_rep2_+T4_t15_2_filter.fastq ./20211105180528381-5936/original/DEFAULT/a10_rep2_+T4_t15_1.fastq ./20211105180528381-5936/original/DEFAULT/a10_rep2_+T4_t15_2.fastq --cores=0 -m 50
Processing paired-end reads on 48 cores ...
Finished in 30.99 s (1 µs/read; 49.66 M reads/minute).

=== Summary ===

Total read pairs processed:         25,647,520
  Read 1 with adapter:               1,257,717 (4.9%)
  Read 2 with adapter:               1,111,663 (4.3%)

== Read fate breakdown ==
Pairs that were too short:               2,327 (0.0%)
Pairs written (passing filters):    25,645,193 (100.0%)

Total basepairs processed: 7,694,256,000 bp
  Read 1: 3,847,128,000 bp
  Read 2: 3,847,128,

  Read 2 with adapter:                 767,348 (3.1%)

== Read fate breakdown ==
Pairs that were too short:               2,806 (0.0%)
Pairs written (passing filters):    24,662,790 (100.0%)

Total basepairs processed: 7,399,678,800 bp
  Read 1: 3,699,839,400 bp
  Read 2: 3,699,839,400 bp
Total written (filtered):  7,387,044,218 bp (99.8%)
  Read 1: 3,692,794,845 bp
  Read 2: 3,694,249,373 bp

=== First read: Adapter 1 ===

Sequence: GATCGGAAGAGCACACGTCTGAACTCCAGTCACGGATGACTATCTCGTATGCCGTCTTCTGCTTG; Type: regular 3'; Length: 65; Trimmed: 832401 times

Minimum overlap: 3
No. of allowed errors:
1-9 bp: 0; 10-19 bp: 1; 20-29 bp: 2; 30-39 bp: 3; 40-49 bp: 4; 50-59 bp: 5; 60-65 bp: 6

Bases preceding removed adapters:
  A: 41.8%
  C: 18.0%
  G: 15.1%
  T: 25.0%
  none/other: 0.2%

Overview of removed sequences
length	count	expect	max.err	error counts
3	460487	385399.9	0	460487
4	132087	96350.0	0	132087
5	37360	24087.5	0	37360
6	14703	6021.9	0	14703
7	9884	1505.5	0	9884
8	8667	376.4	0	8667
9

9	10428	95.1	0	9727 701
10	9978	23.8	1	9138 840
11	9174	5.9	1	8687 487
12	8837	1.5	1	8504 333
13	8422	0.4	1	8158 264
14	7987	0.1	1	7716 271
15	7510	0.0	1	7196 314
16	7255	0.0	1	6999 256
17	7059	0.0	1	6788 271
18	6751	0.0	1	6448 302 1
19	6635	0.0	1	6341 289 5
20	6243	0.0	2	5944 254 45
21	5760	0.0	2	5453 261 46
22	5508	0.0	2	5226 229 53
23	5220	0.0	2	4942 236 42
24	5035	0.0	2	4783 219 33
25	4938	0.0	2	4656 232 50
26	4693	0.0	2	4399 249 45
27	4300	0.0	2	4039 211 50
28	4176	0.0	2	3887 242 46 1
29	4126	0.0	2	3840 249 37
30	4024	0.0	3	3734 221 40 29
31	3667	0.0	3	3409 195 41 22
32	3717	0.0	3	3429 229 42 17
33	3294	0.0	3	3058 190 26 20
34	3237	0.0	3	2983 145 81 28
35	2958	0.0	3	1 2780 144 33
36	2760	0.0	3	0 2581 109 70
37	2834	0.0	3	0 0 2680 109 45
38	2573	0.0	3	0 0 13 2457 103
39	2281	0.0	3	0 0 8 2268 5
40	20	0.0	4	0 0 1 13 6
41	16	0.0	4	0 0 0 7 9
42	64	0.0	4	0 0 0 0 64
43	48	0.0	4	0 0 0 1 47
45	1	0.0	4	0 0 0 0 1
46	1	0.0	4	0 0 0 0 1
53	1	0.0	5	0 0 0 0 0 1
60	1	0.0	6	0 0 0 0 0 1


=== Second

  C: 28.7%
  G: 24.7%
  T: 9.1%
  none/other: 0.0%

Overview of removed sequences
length	count	expect	max.err	error counts
3	393919	429550.1	0	393919
4	107604	107387.5	0	107604
5	33288	26846.9	0	33288
6	15983	6711.7	0	15983
7	7666	1677.9	0	7666
8	6681	419.5	0	6681
9	6862	104.9	0	6148 714
10	6540	26.2	1	5822 718
11	6161	6.6	1	5832 329
12	5630	1.6	1	5292 338
13	5465	0.4	1	5211 254
14	5168	0.1	1	4945 223
15	4850	0.0	1	4642 208
16	4758	0.0	1	4571 187
17	4601	0.0	1	4396 205
18	4271	0.0	1	4006 265
19	4102	0.0	1	3819 273 10
20	4020	0.0	2	3682 285 53
21	3893	0.0	2	3499 328 66
22	3735	0.0	2	3347 323 65
23	3514	0.0	2	3204 263 47
24	3228	0.0	2	2933 249 46
25	3067	0.0	2	2803 219 45
26	2946	0.0	2	2624 274 48
27	2699	0.0	2	2378 283 38
28	2605	0.0	2	2274 286 45
29	2543	0.0	2	2208 277 55 3
30	2411	0.0	3	2055 284 49 23
31	2391	0.0	3	2052 257 48 34
32	2368	0.0	3	2048 249 50 21
33	2153	0.0	3	1853 192 82 26
34	2118	0.0	3	1 1839 181 97
35	1803	0.0	3	0 15 1616 172
36	1539	0.0	3	0 2 75 1462
37	1568	0.0	3	0 0


== Read fate breakdown ==
Pairs that were too short:                 192 (0.0%)
Pairs written (passing filters):    29,907,013 (100.0%)

Total basepairs processed: 8,972,161,500 bp
  Read 1: 4,486,080,750 bp
  Read 2: 4,486,080,750 bp
Total written (filtered):  8,963,522,487 bp (99.9%)
  Read 1: 4,481,241,314 bp
  Read 2: 4,482,281,173 bp

=== First read: Adapter 1 ===

Sequence: GATCGGAAGAGCACACGTCTGAACTCCAGTCACGGATGACTATCTCGTATGCCGTCTTCTGCTTG; Type: regular 3'; Length: 65; Trimmed: 943307 times

Minimum overlap: 3
No. of allowed errors:
1-9 bp: 0; 10-19 bp: 1; 20-29 bp: 2; 30-39 bp: 3; 40-49 bp: 4; 50-59 bp: 5; 60-65 bp: 6

Bases preceding removed adapters:
  A: 27.3%
  C: 24.3%
  G: 17.1%
  T: 31.2%
  none/other: 0.0%

Overview of removed sequences
length	count	expect	max.err	error counts
3	588334	467300.1	0	588334
4	195924	116825.0	0	195924
5	50527	29206.3	0	50527
6	15035	7301.6	0	15035
7	6554	1825.4	0	6554
8	4481	456.3	0	4481
9	5142	114.1	0	4372 770
10	4715	28.5	1	3919 796
11	437

...


Minimum overlap: 3
No. of allowed errors:
1-9 bp: 0; 10-19 bp: 1; 20-29 bp: 2; 30-39 bp: 3; 40-49 bp: 4; 50-59 bp: 5; 60-65 bp: 6

Bases preceding removed adapters:
  A: 40.5%
  C: 18.4%
  G: 14.9%
  T: 26.2%
  none/other: 0.1%

Overview of removed sequences
length	count	expect	max.err	error counts
3	565180	476173.9	0	565180
4	166171	119043.5	0	166171
5	45863	29760.9	0	45863
6	17853	7440.2	0	17853
7	11780	1860.1	0	11780
8	10206	465.0	0	10206
9	10018	116.3	0	9513 505
10	9472	29.1	1	8829 643
11	9175	7.3	1	8651 524
12	8524	1.8	1	8154 370
13	8296	0.5	1	7972 324
14	7490	0.1	1	7135 355
15	7510	0.0	1	7185 325
16	7054	0.0	1	6734 320
17	6598	0.0	1	6329 269
18	6705	0.0	1	6416 287 2
19	6614	0.0	1	6290 321 3
20	6343	0.0	2	5974 293 76
21	6101	0.0	2	5706 313 82
22	5810	0.0	2	5445 304 61
23	5265	0.0	2	4952 245 68
24	4966	0.0	2	4689 225 52
25	4681	0.0	2	4392 234 55
26	4620	0.0	2	4315 244 61
27	4307	0.0	2	4005 247 55
28	4117	0.0	2	3844 217 56
29	3955	0.0	2	3707 199 49
30	3866	0.0	3	3553 241 46 26
31	3

33	2130	0.0	3	1958 127 27 18
34	1978	0.0	3	1815 83 58 22
35	1939	0.0	3	0 1820 65 54
36	1690	0.0	3	0 8 1611 71
37	1641	0.0	3	0 0 13 1627 1
38	1506	0.0	3	0 0 8 1498
39	1463	0.0	3	0 0 0 6 1457
40	1440	0.0	4	0 0 1 1428 11
41	1408	0.0	4	0 0 0 3 1405
42	9	0.0	4	0 0 0 1 8
43	10	0.0	4	0 0 0 1 9
49	1	0.0	4	0 0 0 0 1
50	655	0.0	5	0 0 0 0 0 655
51	589	0.0	5	0 0 0 0 0 589
52	525	0.0	5	0 0 0 0 0 525
53	479	0.0	5	0 0 0 0 0 479
54	483	0.0	5	0 0 0 0 0 483
55	432	0.0	5	0 0 0 0 0 432
56	418	0.0	5	0 0 0 0 0 418
57	385	0.0	5	0 0 0 0 0 385
58	293	0.0	5	0 0 0 0 0 293
59	336	0.0	5	0 0 0 0 0 335 1
60	422	0.0	6	0 0 0 0 0 300 122
61	379	0.0	6	0 0 0 0 0 261 118
62	359	0.0	6	0 0 0 0 0 248 111
63	300	0.0	6	0 0 0 0 0 195 105
64	291	0.0	6	0 0 0 0 0 183 108
65	230	0.0	6	0 0 0 0 0 146 84
66	206	0.0	6	0 0 0 0 0 129 77
67	201	0.0	6	0 0 0 0 0 135 66
68	224	0.0	6	0 0 0 0 0 154 70
69	180	0.0	6	0 0 0 0 0 106 74
70	182	0.0	6	0 0 0 0 0 124 58
71	191	0.0	6	0 0 0 0 0 123 68
72	154	0.0	6	0 0 0 0 0 97 57
73	158	0.0	6	0 0 0 0 0 10

76	88	0.0	6	0 0 0 0 0 48 40
77	96	0.0	6	0 0 0 0 0 49 47
78	89	0.0	6	0 0 0 0 0 50 39
79	60	0.0	6	0 0 0 0 0 31 29
80	68	0.0	6	0 0 0 0 0 35 33
81	75	0.0	6	0 0 0 0 0 41 34
82	61	0.0	6	0 0 0 0 0 23 38
83	51	0.0	6	0 0 0 0 0 33 18
84	43	0.0	6	0 0 0 0 0 22 21
85	48	0.0	6	0 0 0 0 0 23 25
86	46	0.0	6	0 0 0 0 0 24 22
87	41	0.0	6	0 0 0 0 0 26 15
88	32	0.0	6	0 0 0 0 0 15 17
89	33	0.0	6	0 0 0 0 0 15 18
90	34	0.0	6	0 0 0 0 0 15 19
91	30	0.0	6	0 0 0 0 0 15 15
92	26	0.0	6	0 0 0 0 0 15 11
93	23	0.0	6	0 0 0 0 0 10 13
94	15	0.0	6	0 0 0 0 0 8 7
95	16	0.0	6	0 0 0 0 0 10 6
96	18	0.0	6	0 0 0 0 0 9 9
97	12	0.0	6	0 0 0 0 0 5 7
98	8	0.0	6	0 0 0 0 0 2 6
99	11	0.0	6	0 0 0 0 0 6 5
100	11	0.0	6	0 0 0 0 0 5 6
101	7	0.0	6	0 0 0 0 0 3 4
102	6	0.0	6	0 0 0 0 0 5 1
103	9	0.0	6	0 0 0 0 0 3 6
104	11	0.0	6	0 0 0 0 0 5 6
105	7	0.0	6	0 0 0 0 0 5 2
106	4	0.0	6	0 0 0 0 0 1 3
107	6	0.0	6	0 0 0 0 0 1 5
108	7	0.0	6	0 0 0 0 0 2 5
109	1	0.0	6	0 0 0 0 0 0 1
110	2	0.0	6	0 0 0 0 0 0 2
111	2	0.0	6	0 0 0 0 0 1 1
112	1	0.0	6	0 0 0 0 0 0 1


136	2	0.0	6	0 0 0 0 0 0 2
137	5	0.0	6	0 0 0 0 0 0 5
138	5	0.0	6	0 0 0 0 0 0 5
139	9	0.0	6	0 0 0 0 0 0 9
140	11	0.0	6	0 0 0 0 0 0 11
141	5	0.0	6	0 0 0 0 0 0 5
142	11	0.0	6	0 0 0 0 0 0 11
143	10	0.0	6	0 0 0 0 0 0 10
144	16	0.0	6	0 0 0 0 0 0 16
145	4	0.0	6	0 0 0 0 0 0 4
146	12	0.0	6	0 0 0 0 0 0 12
147	3	0.0	6	0 0 0 0 0 0 3
149	13	0.0	6	0 0 0 0 0 0 13
150	201	0.0	6	0 0 0 0 0 1 200


=== Second read: Adapter 2 ===

Sequence: AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGTAGATCTCGGTGGTCGCCGTATCATT; Type: regular 3'; Length: 58; Trimmed: 746388 times

Minimum overlap: 3
No. of allowed errors:
1-9 bp: 0; 10-19 bp: 1; 20-29 bp: 2; 30-39 bp: 3; 40-49 bp: 4; 50-58 bp: 5

Bases preceding removed adapters:
  A: 35.6%
  C: 29.3%
  G: 25.1%
  T: 10.0%
  none/other: 0.0%

Overview of removed sequences
length	count	expect	max.err	error counts
3	389848	404103.2	0	389848
4	106748	101025.8	0	106748
5	35796	25256.5	0	35796
6	19348	6314.1	0	19348
7	10990	1578.5	0	10990
8	9571	394.6	0	9571
9	10202	98.7	0	9371 831
10	9847	

15	16077	0.0	1	15508 569
16	15447	0.0	1	14891 556
17	14858	0.0	1	14285 573
18	14072	0.0	1	13348 721 3
19	13246	0.0	1	12582 657 7
20	13441	0.0	2	12666 684 91
21	12670	0.0	2	11878 703 89
22	11769	0.0	2	11069 616 84
23	11227	0.0	2	10576 550 101
24	10557	0.0	2	9875 588 94
25	9616	0.0	2	8970 568 78
26	9462	0.0	2	8501 853 108
27	8956	0.0	2	8031 814 111
28	8409	0.0	2	7554 745 110
29	8117	0.0	2	7218 766 128 5
30	7786	0.0	3	6936 696 104 50
31	7511	0.0	3	6679 669 118 45
32	7009	0.0	3	6173 686 114 36
33	6627	0.0	3	5862 518 195 52
34	6229	0.0	3	2 5509 552 166
35	5830	0.0	3	0 19 5146 665
36	5483	0.0	3	0 1 4929 553
37	5071	0.0	3	0 1 4533 458 79
38	4880	0.0	3	0 0 3 4372 505
39	4573	0.0	3	0 0 3 4181 389
40	3973	0.0	4	0 0 0 306 3667
41	338	0.0	4	0 0 0 3 335
42	306	0.0	4	0 0 0 0 306
43	287	0.0	4	0 0 0 1 286
44	275	0.0	4	0 0 0 1 274
45	128	0.0	4	0 0 0 0 128
46	48	0.0	4	0 0 0 0 48


### STEP3: run hisat2 

### build the index of reference genome using the .fa format

In [105]:
%%bash
GENOME_file="./yan_results/Escherichia_coli_K12_and_T1_phage_genome.fa"

hisat2-build -p 8 $GENOME_file ./yan_results/genome_index/genome

Settings:
  Output files: "./yan_results/genome_index/genome.*.ht2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Local offset rate: 3 (one in 8)
  Local fTable chars: 6
  Local sequence length: 57344
  Local sequence overlap between two consecutive indexes: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  ./yan_results/Escherichia_coli_K12_and_T1_phage_genome.fa
Reading reference sizes
  Time reading reference sizes: 00:00:00
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:00
  Time to read SNPs and splice sites: 00:00:00
Using parameters --bmax 109253 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters:

Building DifferenceCoverSample
  Building sPrime
  Building sPrimeOrder
  V-Sorting samples
  V-Sorting samples time: 00:00:00
  Allocating rank array
  Ranking v-sort output
  Ranking v-sort output time: 00:00:00
  Invoking Larsson-Sadakane on ranks
  Invoking Larsson-Sadakane on ranks time: 00:00:00
  Sanity-checking and returning
Building samples
Reserving space for 86 sample suffixes
Generating random suffixes
QSorting 86 sample offsets, eliminating duplicates
QSorting sample offsets, eliminating duplicates time: 00:00:00
Multikey QSorting 86 samples
  (Using difference cover)
  Multikey QSorting samples time: 00:00:00
Calculating bucket sizes
Splitting and merging
  Splitting and merging time: 00:00:00
Split 9, merged 39; iterating...
Splitting and merging
  Splitting and merging time: 00:00:00
Split 7, merged 6; iterating...
Splitting and merging
  Splitting and merging time: 00:00:00
Split 4, merged 3; iterating...
Splitting and merging
  Splitting and merging time: 00:00:00
Spl

Converting suffix-array elements to index image
Allocating ftab, absorbFtab
Entering GFM loop


Getting block 1 of 57
  Reserving size (109253) for bucket 1
Getting block 2 of 57
  Reserving size (109253) for bucket 2
  Calculating Z arrays for bucket 1
Getting block 4 of 57
  Reserving size (109253) for bucket 4
Getting block 3 of 57
  Reserving size (109253) for bucket 3
  Calculating Z arrays for bucket 4
Getting block 5 of 57
  Reserving size (109253) for bucket 5
  Entering block accumulator loop for bucket 4:
  Calculating Z arrays for bucket 3
Getting block 6 of 57
  Reserving size (109253) for bucket 6
Getting block 7 of 57
  Reserving size (109253) for bucket 7
  Calculating Z arrays for bucket 5
  Entering block accumulator loop for bucket 3:
  Calculating Z arrays for bucket 2
  Entering block accumulator loop for bucket 1:
  Calculating Z arrays for bucket 7
  Calculating Z arrays for bucket 6
Getting block 8 of 57
  Reserving size (109253) for bucket 8
  Entering block accumulator loop for bucket 6:
  Entering block accumulator loop for bucket 2:
  Calculating Z arra

  bucket 18: 50%
  bucket 17: 50%
Getting block 23 of 57
  Reserving size (109253) for bucket 23
  Calculating Z arrays for bucket 23
  Entering block accumulator loop for bucket 23:
  bucket 15: 90%
  bucket 19: 40%
  bucket 22: 10%
  bucket 18: 60%
  bucket 20: 30%
  bucket 21: 20%
  bucket 17: 60%
  bucket 19: 50%
  bucket 22: 20%
  bucket 23: 10%
  bucket 15: 100%
  Sorting block of length 100722 for bucket 15
  (Using difference cover)
  bucket 18: 70%
  bucket 20: 40%
  bucket 21: 30%
  bucket 17: 70%
  bucket 19: 60%
  bucket 22: 30%
  bucket 23: 20%
  bucket 18: 80%
  bucket 20: 50%
  Sorting block time: 00:00:00
Returning block of 100723 for bucket 15
  bucket 21: 40%
  bucket 17: 80%
  bucket 19: 70%
  bucket 18: 90%
  bucket 22: 40%
Getting block 24 of 57
  Reserving size (109253) for bucket 24
  Calculating Z arrays for bucket 24
  Entering block accumulator loop for bucket 24:
  bucket 23: 30%
  bucket 20: 60%
  bucket 17: 90%
  bucket 21: 50%
  bucket 19: 80%
  bucket 18:

  bucket 38: 30%
Getting block 40 of 57
  Reserving size (109253) for bucket 40
  Calculating Z arrays for bucket 40
  Entering block accumulator loop for bucket 40:
  bucket 39: 20%
  bucket 37: 40%
  bucket 35: 80%
  bucket 36: 70%
  bucket 34: 90%
  Sorting block time: 00:00:00
Returning block of 96954 for bucket 33
  bucket 38: 40%
Getting block 41 of 57
  Reserving size (109253) for bucket 41
  Calculating Z arrays for bucket 41
  Entering block accumulator loop for bucket 41:
  bucket 40: 10%
  bucket 39: 30%
  bucket 37: 50%
  bucket 35: 90%
  bucket 36: 80%
  bucket 34: 100%
  Sorting block of length 82279 for bucket 34
  (Using difference cover)
  bucket 41: 10%
  bucket 38: 50%
  bucket 40: 20%
  bucket 39: 40%
  bucket 35: 100%
  Sorting block of length 55094 for bucket 35
  (Using difference cover)
  bucket 37: 60%
  bucket 36: 90%
  Sorting block time: 00:00:00
Returning block of 82280 for bucket 34
  bucket 41: 20%
Getting block 42 of 57
  Reserving size (109253) for buck

  (Using difference cover)
  bucket 57: 20%
  Sorting block time: 00:00:00
Returning block of 95723 for bucket 49
  bucket 53: 70%
  bucket 52: 80%
  bucket 56: 40%
  bucket 57: 30%
  bucket 54: 60%
  bucket 55: 40%
  Sorting block time: 00:00:00
Returning block of 80126 for bucket 51
  bucket 57: 40%
  bucket 53: 80%
  bucket 56: 50%
  bucket 52: 90%
  bucket 57: 50%
  bucket 55: 50%
  bucket 54: 70%
  bucket 56: 60%
  bucket 57: 60%
  bucket 53: 90%
  bucket 52: 100%
  Sorting block of length 92223 for bucket 52
  (Using difference cover)
  bucket 57: 70%
  bucket 55: 60%
  bucket 56: 70%
  bucket 54: 80%
  bucket 53: 100%
  Sorting block of length 89439 for bucket 53
  (Using difference cover)
  bucket 57: 80%
  Sorting block time: 00:00:00
Returning block of 92224 for bucket 52
  bucket 55: 70%
  bucket 57: 90%
  bucket 56: 80%
  Sorting block time: 00:00:00
Returning block of 89440 for bucket 53
  bucket 54: 90%
  bucket 57: 100%
  Sorting block of length 38235 for bucket 57
  (Us

Exited GFM loop
fchr[A]: 0
fchr[C]: 1150437
fchr[G]: 2332653
fchr[T]: 3513594
fchr[$]: 4661465
Exiting GFM::buildToDisk()
Returning from initFromVector
Wrote 5752637 bytes to primary GFM file: ./yan_results/genome_index/genome.1.ht2
Wrote 1165372 bytes to secondary GFM file: ./yan_results/genome_index/genome.2.ht2
Re-opening _in1 and _in2 as input streams
Returning from GFM constructor
Returning from initFromVector
Wrote 2054631 bytes to primary GFM file: ./yan_results/genome_index/genome.5.ht2
Wrote 1186356 bytes to secondary GFM file: ./yan_results/genome_index/genome.6.ht2
Re-opening _in5 and _in5 as input streams
Returning from HGFM constructor
Headers:
    len: 4661465
    gbwtLen: 4661466
    nodes: 4661466
    sz: 1165367
    gbwtSz: 1165367
    lineRate: 6
    offRate: 4
    offMask: 0xfffffff0
    ftabChars: 10
    eftabLen: 0
    eftabSz: 0
    ftabLen: 1048577
    ftabSz: 4194308
    offsLen: 291342
    offsSz: 1165368
    lineSz: 64
    sideSz: 64
    sideGbwtSz: 48
    sid

### run hisat2 on each filter fastq file, -1 means the reverse sequence file, -2 means the forward sequence file

### note that when distinguish -1 file, -2 file, we need to identify the same suffix of the filter files, here I use the regular expression 'a[0-9]+_.*t.?[0-9]'.

In [26]:
%%bash
# test for the filtering suffix of the file name(success)
fastq_name=$(ls ./20211105180528381-5936/original/DEFAULT/filter/*.fastq | grep -Eo 'a[0-9]+_.*t.?[0-9]' -c | sort | uniq )
echo $fastq_name
    

36


In [27]:
%%bash
##move all the result of the filter file to the filter folder
cd ./20211105180528381-5936/original/DEFAULT/
mv *filter.fastq* ./filter

In [28]:
%%bash
hisat2 --help

HISAT2 version 2.2.1 by Daehwan Kim (infphilo@gmail.com, www.ccb.jhu.edu/people/infphilo)
Usage: 
  hisat2 [options]* -x <ht2-idx> {-1 <m1> -2 <m2> | -U <r> | --sra-acc <SRA accession number>} [-S <sam>]

  <ht2-idx>  Index filename prefix (minus trailing .X.ht2).
  <m1>       Files with #1 mates, paired with files in <m2>.
             Could be gzip'ed (extension: .gz) or bzip2'ed (extension: .bz2).
  <m2>       Files with #2 mates, paired with files in <m1>.
             Could be gzip'ed (extension: .gz) or bzip2'ed (extension: .bz2).
  <r>        Files with unpaired reads.
             Could be gzip'ed (extension: .gz) or bzip2'ed (extension: .bz2).
  <SRA accession number>        Comma-separated list of SRA accession numbers, e.g. --sra-acc SRR353653,SRR353654.
  <sam>      File for SAM output (default: stdout)

  <m1>, <m2>, <r> can be comma-separated lists (no whitespace) and can be
  specified many times.  E.g. '-U file1.fq,file2.fq -U file3.fq'.

Options (defaults in parenthese

  --met-file <path>     send metrics to file at <path> (off)
  --met-stderr          send metrics to stderr (off)
  --met <int>           report internal counters & metrics every <int> secs (1)
  --no-head             suppress header lines, i.e. lines starting with @
  --no-sq               suppress @SQ header lines
  --rg-id <text>        set read group id, reflected in @RG line and RG:Z: opt field
  --rg <text>           add <text> ("lab:value") to @RG line of SAM header.
                        Note: @RG line only printed when --rg-id is set.
  --omit-sec-seq        put '*' in SEQ and QUAL fields for secondary alignments.

 Performance:
  -o/--offrate <int> override offrate of index; must be >= index's offrate
  -p/--threads <int> number of alignment threads to launch (1)
  --reorder          force SAM output order to match order of input reads
  --mm               use memory-mapped I/O for index; many 'hisat2's can share

 Other:
  --qc-filter        filter out reads that are bad a

In [23]:
%%bash
path_hisat2_index="./yan_results/genome_index"
path_fastq="./20211105180528381-5936/original/DEFAULT/filter/"
path_of_result="./yan_results/bam_result/sam_files"
fastq_name=$(ls ./20211105180528381-5936/original/DEFAULT/filter/*.fastq | grep -Eo 'a[0-9]+_.*t.?[0-9]'  | sort | uniq )
for ID in $fastq_name
    do 
    filter_R1=$path_fastq$ID"_1_filter.fastq"
    filter_R2=$path_fastq$ID"_2_filter.fastq"
    hisat2 -p 40 -x $path_hisat2_index"/genome" --rna-strandness RF -1 $filter_R1 -2 $filter_R2 -S $path_of_result"/"$ID".sam"
    done

25645193 reads; of these:
  25645193 (100.00%) were paired; of these:
    970589 (3.78%) aligned concordantly 0 times
    24450969 (95.34%) aligned concordantly exactly 1 time
    223635 (0.87%) aligned concordantly >1 times
    ----
    970589 pairs aligned concordantly 0 times; of these:
      184109 (18.97%) aligned discordantly 1 time
    ----
    786480 pairs aligned 0 times concordantly or discordantly; of these:
      1572960 mates make up the pairs; of these:
        962371 (61.18%) aligned 0 times
        600734 (38.19%) aligned exactly 1 time
        9855 (0.63%) aligned >1 times
98.12% overall alignment rate
24662790 reads; of these:
  24662790 (100.00%) were paired; of these:
    945163 (3.83%) aligned concordantly 0 times
    23443898 (95.06%) aligned concordantly exactly 1 time
    273729 (1.11%) aligned concordantly >1 times
    ----
    945163 pairs aligned concordantly 0 times; of these:
      88103 (9.32%) aligned discordantly 1 time
    ----
    857060 pairs aligned 

    1007107 (3.17%) aligned concordantly 0 times
    30475854 (95.80%) aligned concordantly exactly 1 time
    328771 (1.03%) aligned concordantly >1 times
    ----
    1007107 pairs aligned concordantly 0 times; of these:
      84463 (8.39%) aligned discordantly 1 time
    ----
    922644 pairs aligned 0 times concordantly or discordantly; of these:
      1845288 mates make up the pairs; of these:
        1237851 (67.08%) aligned 0 times
        595828 (32.29%) aligned exactly 1 time
        11609 (0.63%) aligned >1 times
98.05% overall alignment rate
23404625 reads; of these:
  23404625 (100.00%) were paired; of these:
    796911 (3.40%) aligned concordantly 0 times
    22338205 (95.44%) aligned concordantly exactly 1 time
    269509 (1.15%) aligned concordantly >1 times
    ----
    796911 pairs aligned concordantly 0 times; of these:
      94238 (11.83%) aligned discordantly 1 time
    ----
    702673 pairs aligned 0 times concordantly or discordantly; of these:
      1405346 mates

### STEP4: transfer the SAMfiles to BAMfiles, and sort them then set the index
### STEP5:  transfer bam files to bedgraph format, then visualize them on IGV

In [14]:
%%bash 
cd ./yan_results/bam_result/F320/
for i in $(ls /localdisk/home/openngs/openngs/notebooks/ngs/sander/phage_infection_time_course_e_coli_rnaseq/yan_results/bam_result/sam_files/*.sam)
do
    name=$(basename $i .sam)
    samtools view -b -F 320 $i | samtools sort -o $name"_F320_sorted.bam"
    samtools index $name"_F320_sorted.bam"
    bedtools genomecov -ibam $name"_F320_sorted.bam" -bga -strand + > $name"_F320_plus_strand.bedgraph"
    bedtools genomecov -ibam $name"_F320_sorted.bam" -bga -strand - > $name"_F320_minus_strand.bedgraph"
    bedtools genomecov -ibam $name"_F320_sorted.bam" -bga -strand + -5 > $name"_F320_plus_strand_5end.bedgraph" &
    bedtools genomecov -ibam $name"_F320_sorted.bam" -bga -strand - -5 > $name"_F320_minus_strand_5end.bedgraph" &
    bedtools genomecov -ibam $name"_F320_sorted.bam" -bga -strand + -3 > $name"_F320_plus_strand_3end.bedgraph" &
    bedtools genomecov -ibam $name"_F320_sorted.bam" -bga -strand - -3 > $name"_F320_minus_strand_3end.bedgraph" &
    
    #bedtools genomecov -ibam $i -bga -strand + > $i\_plus_strand.bedgraph &
    #bedtools genomecov -ibam $i -bga -strand - > $i\_minus_strand.bedgraph &
    #bedtools genomecov -ibam $i -bga -strand + -5 > $i\_plus_strand_5end.bedgraph &
    #bedtools genomecov -ibam $i -bga -strand - -5 > $i\_minus_strand_5end.bedgraph &
    #bedtools genomecov -ibam $i -bga -strand + -3 > $i\_plus_strand_3end.bedgraph &
    #bedtools genomecov -ibam $i -bga -strand - -3 > $i\_minus_strand_3end.bedgraph &
done

In [28]:
%%bash
cd ./yan_results/bam_result/
#head -100 a10_rep2_+T4_t15.sam
#samtools view -f 64 a10_rep2_+T4_t15.sam | head -n 5
echo "#########################"
#samtools view a10_rep2_+T4_t15.sam | head -n 10
echo "#########################"

samtools view -b -F 320 ./sam_files/a9_rep2_+T4_t10.sam | samtools sort -o ./F320/a9_rep2_+T4_t10_F320_sorted.bam  
samtools index ./F320/a9_rep2_+T4_t10_F320_sorted.bam
bedtools genomecov -ibam ./F320/a9_rep2_+T4_t10_F320_sorted.bam -bga -strand + > ./F320/a9_rep2_+T4_t10_F320_plus_strand.bedgraph
bedtools genomecov -ibam ./F320/a9_rep2_+T4_t10_F320_sorted.bam -bga -strand - > ./F320/a9_rep2_+T4_t10_F320_minus_strand.bedgraph

#########################
#########################


[bam_sort_core] merging from 13 files and 1 in-memory blocks...


### combine all the replicates of the bedgrapph files into one bedgraph file, then use IGV to visualize

In [20]:
%%bash
cd ./yan_results/bam_result/F320/

t20_minus=`ls ./*.bedgraph | grep -Eo 'a[0-9]+_.*T4_t20_F320_minus_strand.bedgraph'`
echo $t20_minus
bedtools unionbedg -i $t20_minus | awk 'OFS="\t" {sum=0; for (col=4; col<=NF; col++) sum += $col; print $1, $2, $3, sum/(NF-4+1); }' > ./mean_combined_t20_minus_strand.bedgraph

t20_plus=`ls ./*.bedgraph | grep -Eo 'a[0-9]+_.*T4_t20_F320_plus_strand.bedgraph'`
echo $t20_plus
bedtools unionbedg -i $t20_plus | awk 'OFS="\t" {sum=0; for (col=4; col<=NF; col++) sum += $col; print $1, $2, $3, sum/(NF-4+1); }' > ./mean_combined_t20_plus_strand.bedgraph

a11_rep2_+T4_t20_F320_minus_strand.bedgraph a17_rep3_+T4_t20_F320_minus_strand.bedgraph a5_rep1_+T4_t20_F320_minus_strand.bedgraph
a11_rep2_+T4_t20_F320_plus_strand.bedgraph a17_rep3_+T4_t20_F320_plus_strand.bedgraph a5_rep1_+T4_t20_F320_plus_strand.bedgraph


In [21]:
%%bash
cd ./yan_results/bam_result/F320/

t5_minus=`ls ./*.bedgraph | grep -Eo 'a[0-9]+_.*T4_t5_F320_minus_strand.bedgraph'`
echo $t5_minus
bedtools unionbedg -i $t5_minus | awk 'OFS="\t" {sum=0; for (col=4; col<=NF; col++) sum += $col; print $1, $2, $3, sum/(NF-4+1); }' > ./mean_combined_t5_minus_strand.bedgraph

t5_plus=`ls ./*.bedgraph | grep -Eo 'a[0-9]+_.*T4_t5_F320_plus_strand.bedgraph'`
echo $t5_plus
bedtools unionbedg -i $t5_plus | awk 'OFS="\t" {sum=0; for (col=4; col<=NF; col++) sum += $col; print $1, $2, $3, sum/(NF-4+1); }' > ./mean_combined_t5_plus_strand.bedgraph

a15_rep3_+T4_t5_F320_minus_strand.bedgraph a2_rep1_+T4_t5_F320_minus_strand.bedgraph a8_rep2_+T4_t5_F320_minus_strand.bedgraph
a15_rep3_+T4_t5_F320_plus_strand.bedgraph a2_rep1_+T4_t5_F320_plus_strand.bedgraph a8_rep2_+T4_t5_F320_plus_strand.bedgraph


In [22]:
%%bash
cd ./yan_results/bam_result/F320/

t10_minus=`ls ./*.bedgraph | grep -Eo 'a[0-9]+_.*T4_t10_F320_minus_strand.bedgraph'`
echo $t10_minus
bedtools unionbedg -i $t10_minus | awk 'OFS="\t" {sum=0; for (col=4; col<=NF; col++) sum += $col; print $1, $2, $3, sum/(NF-4+1); }' > ./mean_combined_t10_minus_strand.bedgraph

t10_plus=`ls ./*.bedgraph | grep -Eo 'a[0-9]+_.*T4_t10_F320_plus_strand.bedgraph'`
echo $t10_plus
bedtools unionbedg -i $t10_plus | awk 'OFS="\t" {sum=0; for (col=4; col<=NF; col++) sum += $col; print $1, $2, $3, sum/(NF-4+1); }' > ./mean_combined_t10_plus_strand.bedgraph

a16_rep3_+T4_t10_F320_minus_strand.bedgraph a3_rep1_+T4_t10_F320_minus_strand.bedgraph a9_rep2_+T4_t10_F320_minus_strand.bedgraph
a16_rep3_+T4_t10_F320_plus_strand.bedgraph a3_rep1_+T4_t10_F320_plus_strand.bedgraph a9_rep2_+T4_t10_F320_plus_strand.bedgraph


In [23]:
%%bash
cd ./yan_results/bam_result/F320/

t15_minus=`ls ./*.bedgraph | grep -Eo 'a[0-9]+_.*T4_t15_F320_minus_strand.bedgraph'`
echo $t15_minus
bedtools unionbedg -i $t15_minus | awk 'OFS="\t" {sum=0; for (col=4; col<=NF; col++) sum += $col; print $1, $2, $3, sum/(NF-4+1); }' > ./mean_combined_t15_minus_strand.bedgraph

t15_plus=`ls ./*.bedgraph | grep -Eo 'a[0-9]+_.*T4_t15_F320_plus_strand.bedgraph'`
echo $t15_plus
bedtools unionbedg -i $t15_plus | awk 'OFS="\t" {sum=0; for (col=4; col<=NF; col++) sum += $col; print $1, $2, $3, sum/(NF-4+1); }' > ./mean_combined_t15_plus_strand.bedgraph

a10_rep2_+T4_t15_F320_minus_strand.bedgraph a4_rep1_+T4_t15_F320_minus_strand.bedgraph
a10_rep2_+T4_t15_F320_plus_strand.bedgraph a4_rep1_+T4_t15_F320_plus_strand.bedgraph


### Download all the bedgraph files

In [24]:
%%bash
cd ./yan_results/bam_result/F320/
#tar cvf combined_bedgraph.tar combined*
tar cvf mean_combined_F320_bedgraph.tar mean_combined*

## then click on the download button of the new bedgraph.tar file

mean_combined_t10_minus_strand.bedgraph
mean_combined_t10_plus_strand.bedgraph
mean_combined_t15_minus_strand.bedgraph
mean_combined_t15_plus_strand.bedgraph
mean_combined_t20_minus_strand.bedgraph
mean_combined_t20_plus_strand.bedgraph
mean_combined_t5_minus_strand.bedgraph
mean_combined_t5_plus_strand.bedgraph


### STEP6:  Run the bedtools to get the raw count of per file, and combine the counts

### Firstly, create a new file named count.file

In [ ]:
pwd

In [ ]:
%%bash
cd ./yan_results/bam_result/F320/
touch count_all.file

In [38]:
%%bash
fastq_name=$(ls ./20211105180528381-5936/original/DEFAULT/filter/*.fastq | grep -Eo 'a[0-9]+_.*t.?[0-9]' | sort | uniq )
echo $fastq_name

a10_rep2_+T4_t15 a11_rep2_+T4_t20 a12_rep2_control_t20 a13_rep3_control_t-5 a14_rep3_+T4_t2 a15_rep3_+T4_t5 a16_rep3_+T4_t10 a17_rep3_+T4_t20 a18_rep3_control_t20 a1_rep1_control_t-5 a2_rep1_+T4_t5 a3_rep1_+T4_t10 a4_rep1_+T4_t15 a5_rep1_+T4_t20 a6_rep1_control_t20 a7_rep2_control_t-5 a8_rep2_+T4_t5 a9_rep2_+T4_t10


In [1]:
%%bash
fastq_name=$(ls ./20211105180528381-5936/original/DEFAULT/filter/*.fastq | grep -Eo 'a[0-9]+_.*t.?[0-9]' | sort | uniq )
command1="Chromosome\tprotein_coding\t5UTR\t4\t189\t2735.0\t+\t.\tid_list\t"
command="bedtools multicov -bams " 
#GTF_file="./yan_results/Escherichia_coli_K12_and_T1_phage_genome.gtf"
#GTF_file="./yan_results/T1.gtf"
GTF_file="./yan_results/E_coli_flattened_T1.gtf"

path_of_result="./yan_results/bam_result/F320"

for ID in $fastq_name
    do
    command+=$path_of_result"/"$ID"_F320_sorted.bam " 
    command1+=$ID"\t"
    done
     command+="-bed $GTF_file > ./yan_results/bam_result/F320/count_new.file"
     #echo -e $command1 > ./yan_results/counts_title.txt
    echo $command
    eval $command
    echo "sed -i '1i"$command1"' ./yan_results/bam_result/F320/count_new.file " > run.sh
    bash run.sh 
    rm run.sh &


bedtools multicov -bams ./yan_results/bam_result/F320/a10_rep2_+T4_t15_F320_sorted.bam ./yan_results/bam_result/F320/a11_rep2_+T4_t20_F320_sorted.bam ./yan_results/bam_result/F320/a12_rep2_control_t20_F320_sorted.bam ./yan_results/bam_result/F320/a13_rep3_control_t-5_F320_sorted.bam ./yan_results/bam_result/F320/a14_rep3_+T4_t2_F320_sorted.bam ./yan_results/bam_result/F320/a15_rep3_+T4_t5_F320_sorted.bam ./yan_results/bam_result/F320/a16_rep3_+T4_t10_F320_sorted.bam ./yan_results/bam_result/F320/a17_rep3_+T4_t20_F320_sorted.bam ./yan_results/bam_result/F320/a18_rep3_control_t20_F320_sorted.bam ./yan_results/bam_result/F320/a1_rep1_control_t-5_F320_sorted.bam ./yan_results/bam_result/F320/a2_rep1_+T4_t5_F320_sorted.bam ./yan_results/bam_result/F320/a3_rep1_+T4_t10_F320_sorted.bam ./yan_results/bam_result/F320/a4_rep1_+T4_t15_F320_sorted.bam ./yan_results/bam_result/F320/a5_rep1_+T4_t20_F320_sorted.bam ./yan_results/bam_result/F320/a6_rep1_control_t20_F320_sorted.bam ./yan_results/bam_re

In [9]:
%%bash
# edit the count file using linux
# sort all the columns 
cd ./yan_results/bam_result/F320
awk '{FS="\t"; OFS="\t"; {print $9,$19,$25,$13,$24,$12,$18,$20,$26,$15,$21,$27,$16,$22,$10,$23,$11,$17;}}' count_new.file > sorted_count_Ecoli_T1_name.csv
#head sorted_count_Ecoli_T1_name.csv
awk '{FS="\t"; OFS=";"; {print $1;}}' sorted_count_Ecoli_T1_name.csv > gene_name.file 

head -4497 gene_name.file > Ecoli_name.file
grep "AY216660" gene_name.file > T1_name.file 

# get rid of all the "", only keep the gene name
cut -d ";" -f 2 Ecoli_name.file |cut -d " " -f 3|tail -n +2| sed 's/\"//g' > Ecoli_gene_name.file

cut -d ";" -f 4 T1_name.file |cut -d " " -f 3| sed 's/\"//g' > T1_gene_name.file

#concatenate the gene name of Ecoli and T1
cat Ecoli_gene_name.file T1_gene_name.file > all_gene_name.file

#add a column name to all the gene
echo "gene_name" > header_name.file
cat header_name.file all_gene_name.file > sum_gene_name.file

cut -f 2- sorted_count_Ecoli_T1_name.csv > no_name.file

paste sum_gene_name.file no_name.file > sorted_count_table.csv

rm *name*


### Result Datasets, they will hold the selected files and current notebook file


In [ ]:
# Details of the analysis, please overwrite defaults and complete the description

title = 'Analysis of Processing_of_fastq_files' # name your analysis
notes = '' # describe/comment this analysis

from obis_connector import NewDataSet 
 
datasets = [] 
 
analysed = NewDataSet( 
    name = title, 
    type = 'ANALYZED_DATA', 
    notes = notes, 
    files = ['folder_with_files','fileName1'], 
    parents = resultDatasetParents 
) 
datasets.append(analysed) 
 
 
# When set to True the uploaded files will be moved into the default location inside working directory 
# that way subsequent analisis of OpenBIS DataSets will not need to download them before processing 
# It is safe to set it to True as the files are not deleted so they can be brought back 
moveFilesOnSuccess = False 
 
 


## Code versions and execution environment information
### You should always call it before uploading data sets to record versions used


In [ ]:
%%bash
# Please only use scripts and annotations files from these folders so they will be tracked with the analysis
SCRIPTS=/homes2/openngs/openngs/scripts2
ANNOTATIONS=/homes2/openngs/openngs/annotation_files2
NGS_TOOLS=/homes2/openngs/openngs/ngs_tools

$SCRIPTS/check_dir_status.sh $SCRIPTS || exit 1
$SCRIPTS/check_dir_status.sh $ANNOTATIONS || exit 1
$SCRIPTS/check_dir_status.sh $NGS_TOOLS || exit 1


In [ ]:
# explicitly imported global modules
import obis_connector

print('global modules:\n')
print('\n'.join(obis_connector.list_imported_modules(globals())))


### Save notebook before triggering upload and set upload = True below!

In [ ]:
# Set upload to True to send data to OpenBIS (remember to save the Notebook first!!!!)
upload = False


#### You should not need to change anything below

In [ ]:
%%javascript

require(["base/js/namespace"],function(Jupyter) {
    Jupyter.notebook.save_checkpoint();
});


In [ ]:
# You should not need to change this cell

import os
import sys
import time
# Result Root node (selected in ELN dialog)
data_root= o.get_object('/OPENNGS/JUP_783')

# leave true to have new analysis container (sample/object) for the results DataSet (recommended),
# otherwise results become directly linked to data_root node you have selected at ELN dialog
new_container = True

if upload:

    # saving current python environment
    list_python_dep = '/homes2/openngs/openngs/scripts2/list_python_dep.sh'
    if not os.path.exists(list_python_dep):
        raise ValueError('Cannot find the environmet listing script')

    dep_file = "python_env.{}.txt".format(time.time())
    !{list_python_dep} {sys.prefix} {dep_file}

    # in try so that the env file is deleted on completion
    try:

        snapshot = obis_connector.save_snapshot(fileName)
        jupyter_files = [fileName, snapshot, dep_file]
        analysis_props = {'name':title, 'notes':notes}
        
        res = obis_connector.upload_analysis_to_obis(datasets, 
                                  data_root = data_root,
                                  jupyter_files = jupyter_files,
                                  workspace = workspace, moveFilesOnSuccess = moveFilesOnSuccess,
                                  new_container = new_container,
                                  object_kind = 'JUPYTER_ANALYSIS', analysis_props = analysis_props)
        for ds in res: 
            print(ds.summary) 
    finally: 
        os.remove(dep_file)
        os.remove(snapshot)
else:
    raise ValueError('Save notebook and change upload to True to store results in OpenBIS')

